In [1]:
import pandas as pd
import numpy as np
import os

#ESM1 preds obtained from: https://www.nature.com/articles/s41588-023-01465-0#Sec2
#https://huggingface.co/spaces/ntranoslab/esm_variants

C:\Users\astar\anaconda3\Lib\site-packages\pandas\core\arrays\masked.py:60: UserWarning: Pandas requires version '1.3.6' or newer of 'bottleneck' (version '1.3.5' currently installed).
  from pandas.core import (


In [2]:
#Reading data (need to modify for 447 PhyloP)
v = pd.read_csv("HumChp_Mis_Final.bed", sep = "\t")
vv = pd.read_csv("HumPoly_Mis_Final.bed", sep = "\t")
vv = vv.drop_duplicates("Position")
v = v[~v["Position"].isin(list(vv["Position"]))]

#Add the 447-way information
v.index = v["Position"]
vv.index = vv["Position"]

p447 = pd.read_csv("447/HumChp_Mis_Final_PhyloP447.txt", sep = "\t").set_index("Position")
v = v.join(p447)
v = v.drop_duplicates("Position")

p447 = pd.read_csv("447/HumPoly_Mis_Final_PhyloP447.txt", sep = "\t").set_index("Position")
vv = vv.join(p447)
vv = vv.drop_duplicates("Position")

#Important parameters
maf_cut = 0.01
spec_sup = 0
pc = -1

print(v.shape)

v = v[v["SpecSup447"] != "."]
vv = vv[vv["SpecSup447"] != "."]

print(v.shape)

v = v[v["PhyloP447"] != "."]
vv = vv[vv["PhyloP447"] != "."]

v["PhyloP447"] = v["PhyloP447"].astype(float)
vv["PhyloP447"] = vv["PhyloP447"].astype(float)


print(v.shape)

v = v[v['SpecSup447'].astype(int) > spec_sup]
vv = vv[vv['SpecSup447'].astype(int) > spec_sup]

print(v.shape)

yvalls = vv[vv["MAFMaxAcrossAncestry"] >= maf_cut]
yvalls = yvalls[yvalls["MAFMaxAcrossAncestry"] <= 1-maf_cut]

#Only keep if we have the essential columns for this analysis
v = v.dropna(subset = ["SpecSup447", "PhyloP447", "Position", "Gene"])
yvalls = yvalls.dropna(subset = ["SpecSup447", "PhyloP447", "Position", "Gene"])

v.index = v["Gene"]
yvalls.index = yvalls["Gene"]

#Remove HLA
out = []
for index, row in v.iterrows():
    if "HLA-" not in index:
        out.append(row)
v = pd.DataFrame(out)

out = []
for index, row in yvalls.iterrows():
    if "HLA-" not in index:
        out.append(row)
yvalls = pd.DataFrame(out)


v = v.drop_duplicates("Position")
yvalls = yvalls.drop_duplicates("Position")
v

(29874, 12)
(29542, 12)
(29542, 12)
(29542, 12)


,Position,Gene,MskHumPhyloP,MskHBCGOPhyloP,SpeciesSupport,470MammalPhastCons,AminoAcidChange,PolyPhen,SIFT,PhyloP447,SpecSup447,PhastCons447
SAMD11,chr1:939293,SAMD11,5.814000,5.788000,235,0.033,R/P,tolerated_low_confidence(0.42),unknown(0),6.836,439,1.0
SAMD11,chr1:942185,SAMD11,1.560000,1.547000,210,1,V/I,tolerated(1),benign(0),0.848,410,1.0
SAMD11,chr1:943879,SAMD11,-0.170000,-0.177000,211,0.001,D/A,tolerated_low_confidence(1),benign(0),-1.080,369,0.0
SAMD11,chr1:944077,SAMD11,-2.362000,-2.384000,228,0,Q/R,tolerated(1),benign(0),-2.269,430,0.0
NOC2L,chr1:952525,NOC2L,5.707000,5.686000,239,1,W/R,tolerated(1),benign(0),0.240,440,0.004
...,...,...,...,...,...,...,...,...,...,...,...,...
F8,chrX:154930412,F8,-3.212000,-3.235000,233,0,R/S,tolerated(0.11),benign(0),-1.617,438,0.000000
F8,chrX:154930465,F8,1.515000,1.500000,236,0.113,Q/E,tolerated(1),benign(0),2.661,441,0.997000
F8,chrX:154931389,F8,0.934000,0.922000,239,0,I/V,tolerated(1),benign(0),1.313,443,0.027000
F8,chrX:154999539,F8,2.661000,2.644000,233,0.909,L/V,tolerated(1),benign(0),4.193,437,1.000000


In [7]:
vx = pd.read_csv("ProteinLevel/ProtVar_HC_Dedup.sort.prot.bed", sep = "\t", header = None).set_index(3)
v["PositionPlus"] = v.index
v.index = v["Position"]
vu = v.join(vx).dropna()
v.index = v["PositionPlus"]
vu.index = vu["Position"]

vvx = pd.read_csv("ProteinLevel/ProtVar_Poly_Dedup.sort.prot.bed", sep = "\t", header = None).set_index(3)
yvalls["PositionPlus"] = yvalls.index
yvalls.index = yvalls["Position"]
yvallsu = yvalls.join(vvx).dropna()
yvalls.index = yvalls["PositionPlus"]
yvallsu.index = yvallsu["Position"]

v = v.drop_duplicates("Position")
yvalls = yvalls.drop_duplicates("Position")

vu = vu.drop_duplicates("Position")
yvallsu = yvallsu.drop_duplicates("Position")


In [8]:
s1 = pd.Series([x[0] for x in vu["AminoAcidChange"].str.split("/")])
s1.index = vu.index
s2 = pd.Series([x[1] for x in vu["AminoAcidChange"].str.split("/")])
s2.index = vu.index
vu.index = vu[0] + "_" + + s1 + vu[2].astype(int).astype(str) + s2
vu

,Position,Gene,MskHumPhyloP,MskHBCGOPhyloP,SpeciesSupport,470MammalPhastCons,AminoAcidChange,PolyPhen,SIFT,PhyloP447,SpecSup447,PhastCons447,PositionPlus,0,1,2,4
Q96NU1_R180P,chr1:939293,SAMD11,5.814000,5.788000,235,0.033,R/P,tolerated_low_confidence(0.42),unknown(0),6.836,439,1.0,SAMD11,Q96NU1,179.0,180.0,canonical
Q96NU1_V307I,chr1:942185,SAMD11,1.560000,1.547000,210,1,V/I,tolerated(1),benign(0),0.848,410,1.0,SAMD11,Q96NU1,306.0,307.0,canonical
H7C3J6_D451A,chr1:943879,SAMD11,-0.170000,-0.177000,211,0.001,D/A,tolerated_low_confidence(1),benign(0),-1.080,369,0.0,SAMD11,H7C3J6,450.0,451.0,fragment
Q96NU1_Q657R,chr1:944077,SAMD11,-2.362000,-2.384000,228,0,Q/R,tolerated(1),benign(0),-2.269,430,0.0,SAMD11,Q96NU1,656.0,657.0,canonical
Q9Y3T9_W360R,chr1:952525,NOC2L,5.707000,5.686000,239,1,W/R,tolerated(1),benign(0),0.240,440,0.004,NOC2L,Q9Y3T9,359.0,360.0,likely_canonical
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
P00451_R1126S,chrX:154930412,F8,-3.212000,-3.235000,233,0,R/S,tolerated(0.11),benign(0),-1.617,438,0.000000,F8,P00451,1125.0,1126.0,canonical
P00451_Q1109E,chrX:154930465,F8,1.515000,1.500000,236,0.113,Q/E,tolerated(1),benign(0),2.661,441,0.997000,F8,P00451,1108.0,1109.0,canonical
P00451_I801V,chrX:154931389,F8,0.934000,0.922000,239,0,I/V,tolerated(1),benign(0),1.313,443,0.027000,F8,P00451,800.0,801.0,canonical
P00451_L69V,chrX:154999539,F8,2.661000,2.644000,233,0.909,L/V,tolerated(1),benign(0),4.193,437,1.000000,F8,P00451,68.0,69.0,canonical


In [9]:
s1 = pd.Series([x[0] for x in yvallsu["AminoAcidChange"].str.split("/")])
s1.index = yvallsu.index
s2 = pd.Series([x[1] for x in yvallsu["AminoAcidChange"].str.split("/")])
s2.index = yvallsu.index
yvallsu.index = yvallsu[0] + "_" + + s1 + yvallsu[2].astype(int).astype(str) + s2
yvallsu

,Position,MAFMaxAcrossAncestry,Gene,MskHumPhyloP,MskHBCGOPhyloP,SpeciesSupport,470MammalPhastCons,AminoAcidChange,PolyPhen,SIFT,PhyloP447,SpecSup447,PhastCons447,PositionPlus,0,1,2,4
A0A2U3U0J3_F134C,chr1:69428,0.011046,OR4F5,5.038000,5.021000,166,0.254,F/C,deleterious(0.02),benign(0.092),6.040,333,1.0,OR4F5,A0A2U3U0J3,133.0,134.0,likely_noncanonical
A0A087WX24_L46H,chr1:924568,0.043020,SAMD11,3.917000,3.903000,201,1,L/H,tolerated_low_confidence(0.29),probably_damaging(0.941),2.697,392,1.0,SAMD11,A0A087WX24,45.0,46.0,likely_noncanonical
Q96NU1_R28Q,chr1:930165,0.010506,SAMD11,8.613000,8.590000,234,1,R/Q,deleterious(0),possibly_damaging(0.842),10.338,437,1.0,SAMD11,Q96NU1,27.0,28.0,canonical
Q96NU1_R41Q,chr1:930204,0.029535,SAMD11,1.044000,1.033000,234,0.001,R/Q,deleterious(0.02),possibly_damaging(0.808),1.541,437,0.259,SAMD11,Q96NU1,40.0,41.0,canonical
Q96NU1_H78Y,chr1:930314,0.116718,SAMD11,3.622000,3.604000,234,0.078,H/Y,deleterious(0.01),possibly_damaging(0.736),2.856,437,0.004,SAMD11,Q96NU1,77.0,78.0,canonical
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
A0A7I2YQU6_P467S,chrX:156025336,0.010092,WASH6P,2.928000,2.910000,223,0.486,P/S,tolerated(0.1),possibly_damaging(0.464),4.478,427,1.000000,WASH6P,A0A7I2YQU6,466.0,467.0,likely_noncanonical
Q9BQ87_S75I,chrY:7063916,0.018229,TBL1Y,8.004000,7.941000,211,1,S/I,deleterious_low_confidence(0),benign(0.026),0.000,101,0.065,TBL1Y,Q9BQ87,74.0,75.0,canonical
Q9BQ87_T125S,chrY:7064065,0.049673,TBL1Y,0.514000,0.496000,203,0.002,T/S,tolerated_low_confidence(0.27),benign(0.213),0.000,101,0.049,TBL1Y,Q9BQ87,124.0,125.0,canonical
Q9BQ87_E146D,chrY:7064130,0.051546,TBL1Y,2.093000,2.054000,223,1,E/D,tolerated_low_confidence(0.07),benign(0.005),0.000,101,0.042,TBL1Y,Q9BQ87,145.0,146.0,canonical


In [14]:
import os
to_build = pd.DataFrame()
c = 0
for file in os.listdir("ALL_hum_isoforms_ESM1b_LLR"):
    c += 1
    if c % 1000 == 0:
        print(c)
    if "-" in file:
        protein = file.split("-")[0]
    else:
        protein = file.split("_")[0]
    if protein in np.unique(vu[0]):
        x = pd.read_csv("ALL_hum_isoforms_ESM1b_LLR/" + file).set_index("Unnamed: 0")
        x = x.reset_index()
        x = pd.melt(x, id_vars='Unnamed: 0', value_vars=x.columns)
        x.index = pd.Series(np.repeat(protein, x.shape[0])) + "_" + x["variable"].str.replace(" ", "") + x["Unnamed: 0"]
        to_build = pd.concat([to_build, vu.join(x).dropna()])
to_build.to_csv("Fixed_ESM1b_Annotated.txt", sep = "\t")

1000
2000
3000
4000
5000
6000
7000
8000
9000
10000
11000
12000
13000
14000
15000
16000
17000
18000
19000
20000
21000
22000
23000
24000
25000
26000
27000
28000
29000
30000
31000
32000
33000
34000
35000
36000
37000
38000
39000
40000
41000
42000


In [15]:
import os
to_build = pd.DataFrame()
c = 0
for file in os.listdir("ALL_hum_isoforms_ESM1b_LLR"):
    c += 1
    if c % 1000 == 0:
        print(c)
    if "-" in file:
        protein = file.split("-")[0]
    else:
        protein = file.split("_")[0]
    if protein in np.unique(yvallsu[0]):
        x = pd.read_csv("ALL_hum_isoforms_ESM1b_LLR/" + file).set_index("Unnamed: 0")
        x = x.reset_index()
        x = pd.melt(x, id_vars='Unnamed: 0', value_vars=x.columns)
        x.index = pd.Series(np.repeat(protein, x.shape[0])) + "_" + x["variable"].str.replace(" ", "") + x["Unnamed: 0"]
        to_build = pd.concat([to_build, yvallsu.join(x).dropna()])
to_build.to_csv("Poly_ESM1b_Annotated.txt", sep = "\t")

1000
2000
3000
4000
5000
6000
7000
8000
9000
10000
11000
12000
13000
14000
15000
16000
17000
18000
19000
20000
21000
22000
23000
24000
25000
26000
27000
28000
29000
30000
31000
32000
33000
34000
35000
36000
37000
38000
39000
40000
41000
42000
